### Introdução

ste conjunto de dados contém 1338 linhas de dados segurados, onde as cobranças de seguro são dadas em relação aos seguintes atributos do segurado: Idade, Sexo, IMC, Número de filhos, Fumante e Região. Os atributos são uma mistura de variáveis ​​numéricas e categóricas.

### Objetivo

Analisar os dados dos clientes e entender como posso utilizar as informações no negócio

### Dicionário do dados

idade - Idade do beneficiário principal
sexo - Gênero do contratante de seguros, feminino / masculino
IMC - Índice de massa corporal, que fornece uma compreensão do corpo, pesos que são relativamente altos ou baixos
crianças - Número de crianças cobertas pelo seguro de saúde / Número de dependentes
fumante - Fumante / Não fumante
região - Area residencial do beneficiário nos EUA: nordeste, sudeste, sudoeste, noroeste
cobranças - Custos médicos individuais cobrados pelo seguro de saúde.

In [ ]:
from pyspark.sql.functions import col, when, sum, avg, row_number
from pyspark.sql.functions import * 



In [ ]:
# instanciando o dataset

dataset = 'dbfs:/FileStore/insurance.csv'

In [ ]:
df_insarunce = spark\
                .read\
                .format("csv")\
                .option("inferschema",True)\
                .option("header",True)\
                .csv(dataset)

df_insarunce.show(5)      

+---+------+----+--------+------+---------+--------+
|age|   sex| bmi|children|smoker|   region|expenses|
+---+------+----+--------+------+---------+--------+
| 19|female|27.9|       0|   yes|southwest|16884.92|
| 18|  male|33.8|       1|    no|southeast| 1725.55|
| 28|  male|33.0|       3|    no|southeast| 4449.46|
| 33|  male|22.7|       0|    no|northwest|21984.47|
| 32|  male|28.9|       0|    no|northwest| 3866.86|
+---+------+----+--------+------+---------+--------+
only showing top 5 rows



In [ ]:
# Exibindo o schema do dataframe
df_insarunce.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- expenses: double (nullable = true)



In [ ]:
#Exibindo as ultimas 5 linhas 

df_insarunce.tail(5)

Out[5]: [Row(age=50, sex='male', bmi=31.0, children=3, smoker='no', region='northwest', expenses=10600.55),
 Row(age=18, sex='female', bmi=31.9, children=0, smoker='no', region='northeast', expenses=2205.98),
 Row(age=18, sex='female', bmi=36.9, children=0, smoker='no', region='southeast', expenses=1629.83),
 Row(age=21, sex='female', bmi=25.8, children=0, smoker='no', region='southwest', expenses=2007.95),
 Row(age=61, sex='female', bmi=29.1, children=0, smoker='yes', region='northwest', expenses=29141.36)]

In [ ]:
#Contando o numero de linhas 
df_insarunce.count()

Out[6]: 1338

In [ ]:
df_insarunce.describe().show()

+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|summary|               age|   sex|               bmi|         children|smoker|   region|          expenses|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+
|  count|              1338|  1338|              1338|             1338|  1338|     1338|              1338|
|   mean| 39.20702541106129|  null|30.665470852017993|  1.0949177877429|  null|     null|13270.422414050803|
| stddev|14.049960379216147|  null|  6.09838219000336|1.205492739781914|  null|     null|12110.011239706473|
|    min|                18|female|              16.0|                0|    no|northeast|           1121.87|
|    max|                64|  male|              53.1|                5|   yes|southwest|          63770.43|
+-------+------------------+------+------------------+-----------------+------+---------+------------------+



In [ ]:
#Verificando a distruição 
df_insarunce.select('smoker').distinct().collect()

Out[8]: [Row(smoker='no'), Row(smoker='yes')]

In [ ]:
#VErificação distruição da coluna sex
df_insarunce.select('sex').distinct().collect()

Out[9]: [Row(sex='female'), Row(sex='male')]

In [ ]:

#Distribuição da região
df_insarunce.select('region').distinct().collect()

Out[10]: [Row(region='northwest'),
 Row(region='southeast'),
 Row(region='northeast'),
 Row(region='southwest')]

In [ ]:
#Verificando missings no dataset
from pyspark.sql.functions import col, sum

df_nulos = df_insarunce.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df_insarunce.columns
    ])

df_nulos.show()

+---+---+---+--------+------+------+--------+
|age|sex|bmi|children|smoker|region|expenses|
+---+---+---+--------+------+------+--------+
|  0|  0|  0|       0|     0|     0|       0|
+---+---+---+--------+------+------+--------+



### Transformação

Nessa etapa irei modificar o nome das colunas e padronizar para ser um dataset com apenas valores numericos




In [ ]:
#Modificando o nome das colunas 
df_insarunce_01 = df_insarunce.withColumnRenamed("age","idade")\
                              .withColumnRenamed("sex","sexo")\
                              .withColumnRenamed("bmi","imc")\
                              .withColumnRenamed("smoker","fumante")\
                              .withColumnRenamed("region", "regiao")\
                              .withColumnRenamed("children","filho")\
                              .withColumnRenamed("expenses","Despesa")

#Validando a modificação
df_insarunce_01.columns


Out[15]: ['idade', 'sexo', 'imc', 'filho', 'fumante', 'regiao', 'Despesa']

In [ ]:
# Transformando a linhas categoricas em linhas numericas
#female (Feminino) = 0
#male (masculino) = 1
from pyspark.sql.functions import regexp_replace

df_insarunce_01 = df_insarunce_01.withColumn('sexo', regexp_replace(df_insarunce_01['sexo'],'female','1'))
df_insarunce_01 = df_insarunce_01.withColumn('sexo', regexp_replace(df_insarunce_01['sexo'],'male','2'))

df_insarunce_01.show(2)



+-----+----+----+-----+-------+---------+--------+
|idade|sexo| imc|filho|fumante|   regiao| Despesa|
+-----+----+----+-----+-------+---------+--------+
|   19|   1|27.9|    0|    yes|southwest|16884.92|
|   18|   2|33.8|    1|     no|southeast| 1725.55|
+-----+----+----+-----+-------+---------+--------+
only showing top 2 rows



In [ ]:
#Transformando as regiões em colunas numericas
#northwest = 0
#southeast = 1
#northeast = 2
#southwest = 3

df_insarunce_01 = df_insarunce_01.withColumn('regiao', regexp_replace(df_insarunce_01['regiao'],'northwest','0'))
df_insarunce_01 = df_insarunce_01.withColumn('regiao', regexp_replace(df_insarunce_01['regiao'],'southeast','1'))
df_insarunce_01 = df_insarunce_01.withColumn('regiao', regexp_replace(df_insarunce_01['regiao'],'northeast','2'))
df_insarunce_01 = df_insarunce_01.withColumn('regiao', regexp_replace(df_insarunce_01['regiao'],'southwest','3'))


#validando a alteração
df_insarunce_01.select('regiao').distinct().collect()

Out[37]: [Row(regiao='3'), Row(regiao='0'), Row(regiao='1'), Row(regiao='2')]

In [ ]:
#Transformando a coluna "fumante"

# no = 0
# yes = 1

df_insarunce_01 = df_insarunce_01.withColumn('fumante', regexp_replace(df_insarunce_01['fumante'],'no','0'))
df_insarunce_01 = df_insarunce_01.withColumn('fumante', regexp_replace(df_insarunce_01['fumante'],'yes','1'))

#Validando a modificação
df_insarunce_01.select('fumante').distinct().collect()

Out[40]: [Row(fumante='0'), Row(fumante='1')]

In [ ]:
#Transformando a coluna IMC

# interpretação do IMC
# Entre 18,5 e 24,9 -	Normal = 0
# Entre 25,0 e 29,9	- Sobrepeso	I = 1
# Entre 30,0 e 39,9 - Obesidade	II = 2
# Maior que 40,0 - Obesidade Grave	III = 3

df_insarunce_01 = df_insarunce_01.withColumn(
    "imc_categoria",
    when((col('imc') >= 18.5) & (col('imc') <= 24.9),'0')
    .when((col('imc') >= 25.0) & (col('imc') <= 29.9),'1')
    .when((col('imc') >= 30.0) & (col('imc') <= 39.9),'2')
    .otherwise(3)
)

df_insarunce_01.select('imc_categoria').distinct().collect()

Out[48]: [Row(imc_categoria='3'),
 Row(imc_categoria='0'),
 Row(imc_categoria='1'),
 Row(imc_categoria='2')]

In [ ]:

#Exibindo o dataframe tratado.
df_insarunce_01.display()

idade,sexo,imc,filho,fumante,regiao,Despesa,imc_categoria
19,1,27.9,0,1,3,16884.92,1
18,2,33.8,1,0,1,1725.55,2
28,2,33.0,3,0,1,4449.46,2
33,2,22.7,0,0,0,21984.47,0
32,2,28.9,0,0,0,3866.86,1
31,1,25.7,0,0,1,3756.62,1
46,1,33.4,1,0,1,8240.59,2
37,1,27.7,3,0,0,7281.51,1
37,2,29.8,2,0,2,6406.41,1
60,1,25.8,0,0,0,28923.14,1


In [ ]:
#Não sei se ficará
df_insarunce_01.printSchema()

root
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- imc: double (nullable = true)
 |-- filho: integer (nullable = true)
 |-- fumante: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- Despesa: double (nullable = true)
 |-- imc_categoria: string (nullable = false)



In [ ]:
from pyspark.sql.types import IntegerType
#conversando coluna string para o tipo integer

df_insarunce_01 = df_insarunce_01.withColumn('sexo', df_insarunce_01['sexo'].cast(IntegerType()))
df_insarunce_01 = df_insarunce_01.withColumn('imc_categoria', df_insarunce_01['imc_categoria'].cast(IntegerType()))
df_insarunce_01 = df_insarunce_01.withColumn('regiao', df_insarunce_01['regiao'].cast(IntegerType()))
df_insarunce_01 = df_insarunce_01.withColumn('fumante', df_insarunce_01['fumante'].cast(IntegerType()))

df_insarunce_01.printSchema()

root
 |-- idade: integer (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- imc: double (nullable = true)
 |-- filho: integer (nullable = true)
 |-- fumante: integer (nullable = true)
 |-- regiao: integer (nullable = true)
 |-- Despesa: double (nullable = true)
 |-- imc_categoria: integer (nullable = true)



In [ ]:
# Transformando o dataframe em tabela sql para realizar consultas

df_insarunce_01.createOrReplaceGlobalTempView('seguro')


### Analise explorátoria

1 - Qual é a maior frequencia de idade entre os clientes ?

2 - Qual a relação entre a quantidade de filhos e o imc ? 

3 - Qual é a media de idade de pessoas fumantes ?

4 - Qual é a região que mais possui clientes ? - ok

5 - Quantas pessoas são fumantes,possuem filho e estão acima do peso normal " Considerando a tabela imc" - ok

6 - Qual a media de despesas gasto por sexo ?

7 - Qual é maior idade do cliente crendenciado ?


In [ ]:
#4 - Qual é a região que mais possui clientes ?

spark.sql("""
          SELECT 
          CASE 
            when regiao = 0 then 'noroeste'
            when regiao = 1 then 'sudeste'
            when regiao = 2 then 'nordeste'
            when regiao = 3 then 'sudoeste'
            end as regiao,
          count(regiao) as quantidade
          FROM global_temp.seguro
          GROUP BY regiao
          ORDER BY quantidade desc
          """).show()


+--------+----------+
|  regiao|quantidade|
+--------+----------+
| sudeste|       364|
|sudoeste|       325|
|noroeste|       325|
|nordeste|       324|
+--------+----------+



In [ ]:
# Qual é o total de despesa gasto por região ?

spark.sql("""
            SELECT 
            CASE 
            when regiao = 0 then 'noroeste'
            when regiao = 1 then 'sudeste'
            when regiao = 2 then 'nordeste'
            when regiao = 3 then 'sudoeste'
            end as regiao,
            round(sum(Despesa),2) as Total_despesas
            FROM global_temp.seguro
            GROUP BY regiao
            order by Total_despesas asc

""").show()

+--------+--------------+
|  regiao|Total_despesas|
+--------+--------------+
|sudoeste|    4012754.82|
|noroeste|    4035711.93|
|nordeste|    4343668.64|
| sudeste|     5363689.8|
+--------+--------------+



In [ ]:
#3 -Quais as idades que mais possuem fumantes ( top 5) ?
spark.sql("""
           
            SELECT idade ,count(idade) as quantidade_fumantes
            from global_temp.seguro
            where fumante = 1
            GROUP BY idade
            ORDER BY quantidade_fumantes desc
            limit 5
""").show()

+-----+-------------------+
|idade|quantidade_fumantes|
+-----+-------------------+
|   19|                 18|
|   43|                 12|
|   18|                 12|
|   47|                 10|
|   20|                  9|
+-----+-------------------+



In [ ]:
#5 - Quantas pessoas são fumantes,possuem filho e estão acima do peso normal " Considerando a tabela imc"

spark.sql("""
           
            SELECT count(fumante) as qts_fumantes
            FROM global_temp.seguro
            where fumante > 0 and imc_categoria > 0 and filho > 0
            """).show()

+------------+
|qts_fumantes|
+------------+
|         132|
+------------+



In [ ]:
#Qual é maior idade do cliente crendenciado ?
spark.sql("""

    SELECT max(idade)
    FROM global_temp.seguro
    
 """).show()

+----------+
|max(idade)|
+----------+
|        64|
+----------+



In [ ]:
# Qual a media de despesas gasto por sexo ?

spark.sql(""" 
          select 
            case
                when sexo = 1 then 'Mulher'
                when sexo = 2 then 'Homen'
            end as sexo,
            round(avg(Despesa),2) as media_despesa
            FROM global_temp.seguro
            group by sexo
          
          """).show()

+------+-------------+
|  sexo|media_despesa|
+------+-------------+
|Mulher|     12569.58|
| Homen|     13956.75|
+------+-------------+

